# Supply chain partitioning example


In this notebook, we will use cudf and cuGraph to show how to read multiple files, compute WCC, and dump to output files

* Created:   01/22/2020

RAPIDS Versions: 0.11.1

Test Hardware
* GV100 32G, CUDA 10.1

Using docker container: rapidsai/rapidsai:cuda10.1-runtime-centos7

## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon. 

In [15]:
# Import needed libraries
import cugraph
import cudf
import os

#### Prep

In [16]:
# Test files
root_dir = '../data/'
output_dir = "../../"
#input_file_base = 'supplyChain-data'
input_file_base = 'Arijit'

In [17]:
def print_components(_df, id, maxColumnLength):
    
    _f = _df.query('labels == @id')
  
    part = []
    for i in range(len(_f)):
        part.append(_f['vertices'][i] + 1)
        
    for i in range(len(_f), maxColumnLength):
        part.append(None)
    print(part)
    
    return part

#### loop to read files, renumber the node list (if index not starting from 0), create graphs, compute WCC, and dump WCC label info to multiple output files

In [20]:
for i in [1,2]:

    datafile = root_dir+input_file_base+str(i)+'.csv'
    gdf = cudf.read_csv(datafile, delimiter=",", names=['src', 'dst'], dtype=['int32', 'int32'], skiprows=1 )
    
    # renumbering in case the node index does not start from 0
    gdf['renumbered_src'], gdf['renumbered_dst'], mapping = cugraph.renumber(gdf['src'], gdf['dst'])
    
    # create undirectred graph
    gpuG = cugraph.Graph()
    gpuG.from_cudf_edgelist(gdf, source='renumbered_src', target='renumbered_dst')
    wcc = cugraph.weakly_connected_components(gpuG)
    
    # back to original vertices before numbering 
    wcc['org_vertices'] = mapping[wcc['vertices']]
    
    label_gby = wcc.groupby('labels')
    maxNodesCountPerComponent = label_gby['org_vertices'].count().max()  
    print("Total number of components found : ", wcc['labels'].unique().count())
    print("Max # of nodes in any of the component : ", maxNodesCountPerComponent)

    # print out
    tempdf = cudf.DataFrame()
    outputfile = output_dir + input_file_base+str(i) + '_output.csv'
    for j in wcc['labels'].unique():
        print("Vertex Ids that belong to component label ", j, ": ") 
        
        tempdf['WCC'+str(j)] = print_components(wcc, j, maxNodesCountPerComponent)
    tempdf.to_csv(outputfile,index=False)

Total number of components found :  1
Max # of nodes in any of the component :  60
Vertex Ids that belong to component label  1 : 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
Total number of components found :  2
Max # of nodes in any of the component :  23
Vertex Ids that belong to component label  1 : 
[1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 32, 33, 34, 35, 36, 37, 38]
Vertex Ids that belong to component label  3 : 
[3, 4, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 39, 40, 41, 42, 43, 44, 45, None]
